In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomRotation(30),
    transforms.Normalize((0.5,), (0.5))
])

In [ ]:
dataset = datasets.ImageFolder(root="data", transform=transform)

In [ ]:
dataset

Dataset ImageFolder
    Number of datapoints: 3985
    Root location: data
    StandardTransform
Transform: Compose(
               ToTensor()
               Grayscale(num_output_channels=1)
               RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
               Normalize(mean=(0.5,), std=0.5)
           )

In [ ]:
totalsize = len(dataset)
train_size = int(0.7 * totalsize)
val_size = int(0.15 * totalsize)
test_size = totalsize - train_size - val_size

In [ ]:
val_size

597

In [ ]:
train_df, val_df, test_df = random_split(dataset,[train_size, val_size, test_size])

In [ ]:
train_loader = DataLoader(train_df, batch_size=32, shuffle=True)
val_loader = DataLoader(train_df, batch_size=32, shuffle=False)
test_loader = DataLoader(train_df, batch_size=32, shuffle=False)

In [ ]:
print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

Train: 2789, Validation: 597, Test: 599


In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 37 * 37)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x